# Image Generation for MNIST using Deep Convolutional Progressive GAN

### Following code shows an implementation of DC-Progressive GAN for the Pokemon dataset. The code is designed in a single class for better execution of respective train and test functions.

In [1]:
# Display time duration after execution of the cell
%%time
# Importing libraries
from __future__ import print_function

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling3D, Conv2D, Conv3D, UpSampling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from scipy.misc import imsave as ims
import matplotlib.pyplot as plt
from PIL import Image
import sys
import os
import glob
import numpy as np
from keras.models import model_from_json
import utils

save_interval = 200

class DCGAN():
    def __init__(self):
        self.img_rows = 100
        self.img_cols = 100
        self.channels = 3

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()
        # Generator upsampling is required for better random machine generation
        model.add(Dense(128 * 25 * 25, activation="relu", input_shape=noise_shape))
        model.add(Reshape((25, 25, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size):

        # Load the dataset
        X_train = utils.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        #X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)


        for epoch in range(epochs+1):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Sample noise and generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch == 0:
                model_json = self.generator.to_json()
                with open("weights/generator.json", "w") as json_file:
                    json_file.write(model_json)
            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)
                gen_name = "weights/gen_" + str(epoch) + ".h5"
                self.generator.save_weights(gen_name)

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        #ims("images/pokemon_%d.png" % epoch,utils.merge(gen_imgs,[3,3]))
        ims('images/pokemon_%d.png'%epoch, utils.merge(gen_imgs,[3,3]))

    def test_imgs(self):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, 100))

        # load json and create model
        json_file = open('weights/generator.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        weightlist = glob.glob('weights/*.h5')
        cnt = 0
        for weight in weightlist:
            # load weights into new model
            loaded_model.load_weights(weight)
            gen_imgs = self.generator.predict(noise)
            # Rescale images 0 - 1
            gen_imgs = 0.5 * gen_imgs + 0.5
            ims('images/test_pokemon_%d.png'%cnt, utils.merge(gen_imgs,[3,3]))
            cnt = cnt+save_interval


Using TensorFlow backend.


Wall time: 2.58 s


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
if __name__ == '__main__':
    if not os.path.exists('images/'):
        os.makedirs('images/')

    if not os.path.exists('weights/'):
        os.makedirs('weights/')

In [4]:
%%time
dcgan = DCGAN()
#call train function for network initialization and model training
dcgan.train(epochs=200, batch_size=16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 26, 26, 64)        0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 26, 26, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 26, 26, 64)        0         
__________

65 [D loss: 0.059893, acc.: 100.00%] [G loss: 3.829294]
66 [D loss: 0.024061, acc.: 100.00%] [G loss: 4.524200]
67 [D loss: 0.019704, acc.: 100.00%] [G loss: 5.230810]
68 [D loss: 0.008194, acc.: 100.00%] [G loss: 5.467308]
69 [D loss: 0.043568, acc.: 100.00%] [G loss: 3.222866]
70 [D loss: 0.090147, acc.: 100.00%] [G loss: 2.780261]
71 [D loss: 0.143623, acc.: 93.75%] [G loss: 6.451217]
72 [D loss: 0.227672, acc.: 93.75%] [G loss: 3.165343]
73 [D loss: 0.026552, acc.: 100.00%] [G loss: 4.292233]
74 [D loss: 0.142790, acc.: 100.00%] [G loss: 5.736993]
75 [D loss: 0.166926, acc.: 87.50%] [G loss: 3.787616]
76 [D loss: 0.174505, acc.: 100.00%] [G loss: 2.152539]
77 [D loss: 0.108763, acc.: 93.75%] [G loss: 7.493873]
78 [D loss: 0.591238, acc.: 81.25%] [G loss: 0.132409]
79 [D loss: 1.178607, acc.: 50.00%] [G loss: 12.839725]
80 [D loss: 1.063779, acc.: 62.50%] [G loss: 8.024033]
81 [D loss: 0.169318, acc.: 100.00%] [G loss: 5.849035]
82 [D loss: 1.111605, acc.: 50.00%] [G loss: 10.272861

In [5]:
#Check the respective weights and generate images to save it in images directory
dcgan.test_imgs()